In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import os
plt.rcParams['font.family'] = 'AppleGothic' 
plt.rcParams['axes.unicode_minus'] = False
import koreanize_matplotlib

In [2]:
votes_df = pd.read_parquet("/home/sprintda05/final/parquet/accounts_userquestionrecord.parquet")
users_df = pd.read_parquet("/home/sprintda05/final/parquet/accounts_user.parquet")
school_df = pd.read_parquet("/home/sprintda05/final/parquet/accounts_group.parquet")

In [3]:
reports_df = pd.read_parquet("/home/sprintda05/final/parquet/accounts_timelinereport.parquet")

In [ ]:
# 1. 기본 데이터프레임
votes_df['created_at'] = pd.to_datetime(votes_df['created_at'])

# 1단계: 기간 필터링
qr_step1 = votes_df[
    (votes_df['created_at'] >= '2023-05-01') &
    (votes_df['created_at'] <= '2023-08-01')
]
print("기간 필터만 적용한 user_id 수:", qr_step1['user_id'].nunique())  

기간 필터만 적용한 user_id 수: 4835


accounts_userquestionrecord 테이블에서 created_at 기준으로 기간을 5~7월로 제한

질문을 보낸 유저(user_id) 중 해당 기간에 활동한 사람만 추출

 결과: 4835명 (전체 기간 중 투표를 생성한 유저 수)

In [ ]:
# 2단계: status != 'B'
qr_step2 = qr_step1[qr_step1['status'] != 'B']
print("차단 제외한 user_id 수:", qr_step2['user_id'].nunique())  

차단 제외한 user_id 수: 4834


status == 'B'는 차단된 질문 → 품질 낮은 질문이거나 문제 요소 있음

분석에서 제외하여 신뢰 가능한 투표만 유지

결과: 4834명

In [ ]:
# 신고당한 유저 목록 추출
reported_users = reports_df['reported_user_id'].dropna().unique()

# 신고당한 유저 제거
qr_step3 = qr_step2[~qr_step2['user_id'].isin(reported_users)]
print("신고당한 유저 제외한 user_id 수:", qr_step3['user_id'].nunique())  


신고당한 유저 제외한 user_id 수: 4722


accounts_timelinereport 테이블에서 신고 대상인 reported_user_id를 가져옴

신고당한 유저는 악성 행위 가능성 있음 → 분석에서 제외

user_id가 아니라 reported_user_id가 기준

 4722명 (정상적이고 신뢰할 수 있는 유저 수)

In [ ]:
# 4단계: 최종 필터 (예: answer_status 등)
# 여기서는  answer_status 필터 안 넣고 4722 나오는지 확인
print(" 최종 user_id 수:", qr_step3['user_id'].nunique())  

 최종 user_id 수: 4722


In [15]:
# 종속성 검증
qr_user_ids = qr_step1['user_id'].dropna().unique()
accounts_user_ids = users_df['id'].dropna().unique()

is_subset = set(qr_user_ids).issubset(set(accounts_user_ids))
print(" questionrecord의 user_id는 accounts_user에 포함되는가?", is_subset)

 questionrecord의 user_id는 accounts_user에 포함되는가? True


In [20]:
# accounts_user 중 reported 포함 여부 비교
user_with_report = users_df[users_df['id'].isin(reported_users)]
user_without_report = users_df[~users_df['id'].isin(reported_users)]

print(" accounts_user에서 신고된 유저 수:", user_with_report['id'].nunique())  
print(" 신고 제외한 유저 수:", user_without_report['id'].nunique())
print(" 전체 유저 수:", users_df['id'].nunique())

 accounts_user에서 신고된 유저 수: 125
 신고 제외한 유저 수: 676960
 전체 유저 수: 677085


# 최종 투표율

In [ ]:
# group_id → school_id 매핑
school_df_renamed = school_df.rename(columns={'id': 'group_id'})

# group_id 있는 유저 필터링 + 가입일 필터링
users_df['created_at'] = pd.to_datetime(users_df['created_at'])
user_notna_df = users_df[users_df['group_id'].notna()].copy()
user_filtered_df = user_notna_df[user_notna_df['created_at'] < '2023-08-01']

# school_id 병합
school_user_df = user_filtered_df.merge(
    school_df_renamed[['group_id', 'school_id']],
    on='group_id',
    how='left'
)

# 실제 투표 발생한 10개 학교 ID
valid_school_ids = [271, 369, 5491, 352, 4516, 4426, 1478, 5520, 5372, 1719]

# 해당 학교 소속 유저만 필터링
target_schoo = school_user_df[school_user_df['school_id'].isin(valid_school_ids)].copy()
l_users_df
# 유저 가입 날짜에서 일자만 추출
target_school_users_df['date'] = target_school_users_df['created_at'].dt.date


In [7]:
# 일별 가입자 수 → 누적합으로 분모 계산
user_daily_join_df = (
    target_school_users_df.groupby('date')['id']
    .count()
    .cumsum()
    .reset_index(name='total_users')
)

In [ ]:
# votes_df 날짜 처리
votes_df['created_at'] = pd.to_datetime(votes_df['created_at'])

# 5월 1일 ~ 7월 31일 필터링
votes_5to7_df = votes_df[
    (votes_df['created_at'] >= '2023-05-01') &
    (votes_df['created_at'] <= '2023-08-01')
].copy()

# 10개 학교 소속 유저만 필터링
valid_user_ids = target_school_users_df['id'].unique()
votes_5to7_df = votes_5to7_df[votes_5to7_df['user_id'].isin(valid_user_ids)].copy()

# 날짜 컬럼 생성
votes_5to7_df['date'] = votes_5to7_df['created_at'].dt.date

# 일별 고유 투표 유저 수
vote_daily_df = (
    votes_5to7_df.groupby('date')['user_id']
    .nunique()
    .reset_index(name='voted_users')
)


In [20]:
# 날짜 기준 병합
merged_df = pd.merge(vote_daily_df, user_daily_join_df, on='date', how='left')
# ✅ 8. 누적 가입자 수 결측값은 전날 값으로 채우기 (ffill)
merged_df['total_users'] = merged_df['total_users'].ffill()

# 투표율 계산
merged_df['vote_rate'] = merged_df['voted_users'] / merged_df['total_users']
merged_df['vote_rate_percent'] = (merged_df['vote_rate'] * 100).round(2).astype(str) + '%'


In [18]:
pd.set_option("display.max_rows", None)

In [21]:
merged_df

,date,voted_users,total_users,vote_rate,vote_rate_percent
0,2023-05-01,325,446.0,0.728700,72.87%
1,2023-05-02,413,545.0,0.757798,75.78%
2,2023-05-03,756,934.0,0.809422,80.94%
3,2023-05-04,820,1118.0,0.733453,73.35%
4,2023-05-05,1248,1552.0,0.804124,80.41%
5,2023-05-06,1375,1830.0,0.751366,75.14%
6,2023-05-07,1355,1984.0,0.682964,68.3%
7,2023-05-08,1295,2151.0,0.602046,60.2%
8,2023-05-09,1318,2324.0,0.567126,56.71%
9,2023-05-10,1372,2516.0,0.545310,54.53%
